# Notebook com finalidade de **estudo** para analise de dados e criação de IA com **TensorFlow**, utilizando **Layers Convolution Model**.
###### **Video base: https://www.youtube.com/watch?v=WvoLTXIjBYU**


###### A IA baseia-se em um dataset matricial formado por jogadas aleatórias criadas pela biblioteca do Stockfish.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install python-chess==0.31.3

In [ ]:
import chess
import chess.engine
import random
import numpy

#caminho para os arquivos para utilizar o anaconda.
#env = r'C:\Users\vpera\Desktop\CHESS AI'

# função que cria uma jogada aleatória para testar o visual.
def random_board(max_depth=200):
  board = chess.Board()
  depth = random.randrange(0, max_depth)

  for _ in range(depth):
    all_moves = list(board.legal_moves)
    random_move = random.choice(all_moves)
    board.push(random_move)
    if board.is_game_over():
      break

  return board


# função que cria o score baseado no stockfish.
    # se negativo a vantagem é das negras;
    # se positivo a vantagem é das brancas;

#caminho para encontrar o stockfish (UTILIZAR STOCKFISH LINUX CASO ESTEJA NA VM DO GOOGLE COLAB)
enginePath = '/content/drive/MyDrive/DATA_SCIENCE/CHESS_IA/stockfish_linux'

##Resultado baseado no score das brancas (negativo é vantagem das negras)
def stockfish(board, depth):
  with chess.engine.SimpleEngine.popen_uci(enginePath) as sf:
    result = sf.analyse(board, chess.engine.Limit(depth=depth))
    score = result['score'].white().score()
    return score
  
board = random_board()
board

In [ ]:
#Permissão de executavel no google colab
!chmod 755 -R /content/drive/MyDrive/DATA_SCIENCE/CHESS_IA/stockfish_linux

In [ ]:
print(stockfish(board, 10))

# Criando dataset com as jogadas

Baseado na análise **"layer convolution"** do tensor flow, teremos uma matriz 3d **14 x 8 x 8** (NHWC), onde:

14 é a representação da posição de cada peça para negras e brancas.

In [ ]:
#eixo x e y do tabuleiro:
squares_index = {
  'a': 0,
  'b': 1,
  'c': 2,
  'd': 3,
  'e': 4,
  'f': 5,
  'g': 6,
  'h': 7
}


# cria um "index" para cada casa do tabuleiro e assinala a posição:
    #ex: h1 (1) -> (7,1);
def square_to_index(square):
  letter = chess.square_name(square)
  return 8 - int(letter[1]), squares_index[letter[0]]


def split_dims(board):
  # cria matrix 3d do tabuleiro 14x8x8
  board3d = numpy.zeros((14, 8, 8), dtype=numpy.int8)

  # adiciona as peças ao tabuleiro
  for piece in chess.PIECE_TYPES:
    for square in board.pieces(piece, chess.WHITE):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece - 1][7 - idx[0]][idx[1]] = 1
    for square in board.pieces(piece, chess.BLACK):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece + 5][7 - idx[0]][idx[1]] = 1

  # computa as somente jogadas legais ao modelo
  aux = board.turn
  board.turn = chess.WHITE
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[12][i][j] = 1
  board.turn = chess.BLACK
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[13][i][j] = 1
  board.turn = aux

  return board3d

In [ ]:
#printa a partida no momento (basedo no random_board())
split_dims(board)

# TensorFlow Layer Convulation Model



In [ ]:
#executar somente se utilizar anaconda (skipar erro com pydot)
import os
os.environ["PATH"] += os.pathsep + r'C:\Program Files\Graphviz\bin'

In [ ]:
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import tensorflow.keras.utils as utils
import tensorflow.keras.optimizers as optimizers


def build_model(conv_size, conv_depth):
  board3d = layers.Input(shape=(14, 8, 8))

  # Convolutional layers
  x = board3d
  for _ in range(conv_depth):
    x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same', activation='relu')(x)
  x = layers.Flatten()(x)
  x = layers.Dense(64, 'relu')(x)
  x = layers.Dense(1, 'sigmoid')(x)

  return models.Model(inputs=board3d, outputs=x)

In [ ]:
model = build_model(32, 4) #dimensão e profundidade
utils.plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=False)

# TREINO DA IA COM TENSORFLOW UTILIZANDO CALLBACKS PARA STOPAR EM UM NIVEL OTIMO

In [ ]:
import tensorflow.keras.callbacks as callbacks
import numpy as np

def get_dataset():
	container = numpy.load('/content/drive/MyDrive/DATA_SCIENCE/CHESS_IA/dataset.npz')
	b, v = container['b'], container['v']
	v = numpy.asarray(v / abs(v).max() / 2 + 0.5, dtype=numpy.float32) #normaliza o dataset
	return b, v

#divide a base de treino
x_train, y_train = get_dataset()
print(x_train.shape)
print(y_train.shape)

In [ ]:
#compila o modelo
model.compile(optimizer=optimizers.Adam(5e-4), loss='mean_squared_error')
model.summary()
model.fit(x_train, y_train,
          batch_size=2048,
          epochs=1000,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4)])

model.save('model.h5')

#IA vs STOCKFISH

In [ ]:
# Algoritimo minimax de xadrez
def minimax_eval(board):
  board3d = split_dims(board)
  board3d = numpy.expand_dims(board3d, 0)
  return model.predict(board3d)[0][0]


def minimax(board, depth, alpha, beta, maximizing_player):
  if depth == 0 or board.is_game_over():
    return minimax_eval(board)
  
  if maximizing_player:
    max_eval = -numpy.inf
    for move in board.legal_moves:
      board.push(move)
      eval = minimax(board, depth - 1, alpha, beta, False)
      board.pop()
      max_eval = max(max_eval, eval)
      alpha = max(alpha, eval)
      if beta <= alpha:
        break
    return max_eval
  else:
    min_eval = numpy.inf
    for move in board.legal_moves:
      board.push(move)
      eval = minimax(board, depth - 1, alpha, beta, True)
      board.pop()
      min_eval = min(min_eval, eval)
      beta = min(beta, eval)
      if beta <= alpha:
        break
    return min_eval


# Função que faz as jogadas
def get_ai_move(board, depth):
  max_move = None
  max_eval = -numpy.inf

  for move in board.legal_moves:
    board.push(move)
    eval = minimax(board, depth - 1, -numpy.inf, numpy.inf, False)
    board.pop()
    if eval > max_eval:
      max_eval = eval
      max_move = move
  
  return max_move

In [ ]:
#Modelo jogando contra o stockfish
board = chess.Board()

with chess.engine.SimpleEngine.popen_uci(enginePath) as engine:
  while True:
    move = get_ai_move(board, 1)
    board.push(move)
    print(f'\n{board}')
    if board.is_game_over():
      break
      
    move = engine.analyse(board, chess.engine.Limit(time=1), info=chess.engine.INFO_PV)['pv'][0]
    board.push(move)
    print(f'\n{board}')
    if board.is_game_over():
      break

In [ ]:
!chmod +x /content/drive/MyDrive/DATA_SCIENCE/CHESS_IA/stockfish.exe

In [ ]:
import random
random.seed(37)